In [2]:
import gzip
import pandas as pd
import urllib.request
import requests

In [2]:
url = "https://ftp.ncbi.nlm.nih.gov/pub/lu/PubTatorCentral/bioconcepts2pubtatorcentral.offset.gz"
file_path = "bioconcepts2pubtatorcentral.offset.gz"

# Download the file
# urllib.request.urlretrieve(url, file_path)


In [3]:
# Unzip the file
with gzip.open(file_path, 'rb') as f:
    df = pd.read_csv(f, delimiter="\t", header=None, names=['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6'])

df

,Column1,Column2,Column3,Column4,Column5,Column6
0,36915321|t|Schmorl's nodes are associated with...,NaN,NaN,NaN,NaN,NaN
1,36915321|a|Background Schmorl's node (SN) corr...,NaN,NaN,NaN,NaN,NaN
2,36915321,161.0,164.0,men,Species,9606
3,36915321,46.0,77.0,osteoporotic vertebral fracture,Disease,MESH:D058866
4,36915321,161.0,164.0,men,Species,9606
...,...,...,...,...,...,...
1985313874,49084,83.0,92.0,carcinoma,Disease,MESH:D009369
1985313875,49084,185.0,197.0,Methotrexate,Chemical,MESH:D008727
1985313876,49084,327.0,333.0,tumour,Disease,MESH:D009369
1985313877,49085|t|Criteria for an acceptable thromboplas...,NaN,NaN,NaN,NaN,NaN


In [10]:
df2 = df.head(100000)

In [13]:
pattern = r'\d+\|t\||\d+\|a\|' 
result = df[df['Column1'].str.contains(pattern)][['Column1']]

In [ ]:
result.to_csv('output.csv', index=False)


In [3]:
t_pmids = pd.read_csv('teeth_pmids.txt', header=None)[0].tolist()
t_pmids

['31995917',
 '18810139',
 '11630834',
 '11628701',
 '10257004',
 '10253337',
 '10249335',
 '7471157',
 '7468959',
 '7468637',
 '7464309',
 '7453561',
 '7451769',
 '7446456',
 '7349792',
 '7348704',
 '7348560',
 '7347798',
 '7347011',
 '7347009',
 '7345955',
 '7345136',
 '7344451',
 '7344450',
 '7344449',
 '7344448',
 '7344446',
 '7343673',
 '7342897',
 '7342794',
 '7342559',
 '7341645',
 '7340259',
 '7338553',
 '7338550',
 '7338332',
 '7337363',
 '7337359',
 '7337357',
 '7337356',
 '7337354',
 '7337021',
 '7337020',
 '7337014',
 '7337013',
 '7335645',
 '7333876',
 '7331901',
 '7331761',
 '7331160',
 '7330686',
 '7330685',
 '7329980',
 '7327219',
 '7324616',
 '7319423',
 '7319421',
 '7318737',
 '7314803',
 '7308607',
 '7307062',
 '7306381',
 '7302315',
 '7301853',
 '7299519',
 '7298385',
 '7293722',
 '7293554',
 '7291954',
 '7289873',
 '7289863',
 '7289062',
 '7279090',
 '7274862',
 '7274832',
 '7274527',
 '7270296',
 '7270295',
 '7270294',
 '7270293',
 '7270292',
 '7270291',
 '7270290

In [4]:
pmids = t_pmids


In [12]:
import pandas as pd

df_results = pd.DataFrame()  # Define the df_results variable
base_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator?pmids={}&concepts=Bioconcepts"
# Iterate over the list of PMIDs
for pmid in pmids:
    # Fetch information for the current PMID
    url = base_url.format(pmid)
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Process the response data
        data = response.text.split('\n')  # Split the response by newline character
        
        # Check if the response data is empty
        if len(data) >= 2:
            # Extract relevant information
            title = data[0].split('|')[2].strip()  # Title is at index 2 after splitting by '|'
            abstract = data[1].split('|')[2].strip()  # Abstract is at index 2 after splitting by '|'
            
            # Append the information to the dataframe
            df_results = pd.concat([df_results, pd.DataFrame({'PubMed ID': [pmid], 'Title': [title], 'Abstract': [abstract]})], ignore_index=True)
        else:
            print("No data found for PMID:", pmid)
    else:
        print("Error fetching data for PMID:", pmid)

# Print the dataframe
print(df_results)

   PubMed ID                                              Title  \
0   31995917  Use of primates for wear testing of dental res...   
1   18810139  Displaced disc preventing mandibular condyle t...   
2   11630834  Koniag Eskimo tooth ablation: was Hrdlicka rig...   
3   11628701  [The ward for diseases of the jaw and teeth at...   
4   10257004  The demand for dental care services, by income...   
..       ...                                                ...   
95   7249039  Palate morphogenesis. VI. Identification of st...   
96   7236832  Influence of variation in contact stress on th...   
97   7236829  Adsorptive bonding to hydroxyapatite I: adsorp...   
98   7234974           Morphological features of Jat dentition.   
99   7232267  Normal EEG of the restrained twenty-four-hour-...   

                                             Abstract  
0   The mechanical loss of substance, both of teet...  
1   The temporomandibular articulation is briefly ...  
2                          

In [5]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor

def fetch_data(pmid):
    base_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator?pmids={}&concepts=Bioconcepts"
    url = base_url.format(pmid)
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.text.split('\n')
        if len(data) >= 2:
            title = data[0].split('|')[2].strip()
            abstract = data[1].split('|')[2].strip()
            return {'PubMed ID': pmid, 'Title': title, 'Abstract': abstract}
        else:
            print("No data found for PMID:", pmid)
    else:
        print("Error fetching data for PMID:", pmid)
    return None


# Fetch data in parallel
with ThreadPoolExecutor(max_workers=17) as executor:  # You can adjust the max_workers parameter based on your system resources
    results = list(executor.map(fetch_data, pmids))

# Filter out None values (failed requests)
results = [result for result in results if result is not None]

# Create DataFrame from results
df_results = pd.DataFrame(results)

# Print the DataFrame
print(df_results)


Error fetching data for PMID: 7007935
Error fetching data for PMID: 7007623
Error fetching data for PMID: 6953658
Error fetching data for PMID: 6953454
Error fetching data for PMID: 6945747
Error fetching data for PMID: 6938338
Error fetching data for PMID: 6937402
Error fetching data for PMID: 6937468
Error fetching data for PMID: 6942959
Error fetching data for PMID: 6942056
Error fetching data for PMID: 6941987
Error fetching data for PMID: 6941128
Error fetching data for PMID: 6935816
Error fetching data for PMID: 6935799
Error fetching data for PMID: 6935601
Error fetching data for PMID: 6934175
Error fetching data for PMID: 6933197
Error fetching data for PMID: 6933190
Error fetching data for PMID: 6933177
Error fetching data for PMID: 6932903
Error fetching data for PMID: 6932894
Error fetching data for PMID: 6302604
Error fetching data for PMID: 7003092
Error fetching data for PMID: 7003090
Error fetching data for PMID: 6450393
Error fetching data for PMID: 6160105
Error fetchi

KeyboardInterrupt: 

In [7]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def fetch_data(pmid):
    base_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator?pmids={}&concepts=Bioconcepts"
    url = base_url.format(pmid)
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.text.split('\n')
        if len(data) >= 2:
            title = data[0].split('|')[2].strip()
            abstract = data[1].split('|')[2].strip()
            return {'PubMed ID': pmid, 'Title': title, 'Abstract': abstract}
        else:
            print("No data found for PMID:", pmid)
    else:
        print("Error fetching data for PMID:", pmid)
    return None

# Define batch size
batch_size = 1000

# Split pmids into batches
pmids_batches = [pmids[i:i+batch_size] for i in range(0, len(pmids), batch_size)]

# Fetch data in parallel with tqdm progress tracking
results = []
with ThreadPoolExecutor(max_workers=100) as executor:
    for pmids_batch in tqdm(pmids_batches):
        batch_results = list(executor.map(fetch_data, pmids_batch))
        batch_results = [result for result in batch_results if result is not None]
        results.extend(batch_results)

# Create DataFrame from results
df_results = pd.DataFrame(results)

# Print the DataFrame
print(df_results)


  0%|          | 0/346 [00:00<?, ?it/s]

Error fetching data for PMID: 7342897
Error fetching data for PMID: 7249039
Error fetching data for PMID: 7330686
Error fetching data for PMID: 7324616
Error fetching data for PMID: 7270288
Error fetching data for PMID: 11630834
Error fetching data for PMID: 7345955
Error fetching data for PMID: 7319423
Error fetching data for PMID: 7270292
Error fetching data for PMID: 7344449
Error fetching data for PMID: 11628701
Error fetching data for PMID: 7341645
Error fetching data for PMID: 7270290
Error fetching data for PMID: 7257724
Error fetching data for PMID: 7293554
Error fetching data for PMID: 7347798
Error fetching data for PMID: 7306381
Error fetching data for PMID: 7270294
Error fetching data for PMID: 7451769
Error fetching data for PMID: 7236829
Error fetching data for PMID: 10257004
Error fetching data for PMID: 7342794
Error fetching data for PMID: 7337013
Error fetching data for PMID: 7468959
Error fetching data for PMID: 7232267
Error fetching data for PMID: 7307062
Error fet

  0%|          | 1/346 [01:16<7:17:35, 76.10s/it]

Error fetching data for PMID:Error fetching data for PMID: 6934202
 6783568
Error fetching data for PMID: 6450544
Error fetching data for PMID: 6783573
Error fetching data for PMID: 6935162
Error fetching data for PMID: 6934201
Error fetching data for PMID: 6937039
Error fetching data for PMID: 6163795
Error fetching data for PMID: 7444686
Error fetching data for PMID: 6938359
Error fetching data for PMID: 6264586
Error fetching data for PMID: 6452424
Error fetching data for PMID: 6937040
Error fetching data for PMID: 6938361
Error fetching data for PMID: 6163996
Error fetching data for PMID: 6934786
Error fetching data for PMID: 7002217
Error fetching data for PMID: 6934790
Error fetching data for PMID: 6783577
Error fetching data for PMID: 7427747
Error fetching data for PMID: 6779232
Error fetching data for PMID: 6783574
Error fetching data for PMID: 6164645
Error fetching data for PMID: 6777489
Error fetching data for PMID: 6164715
Error fetching data for PMID: 10248896
Error fetch

  1%|          | 2/346 [02:37<7:35:51, 79.51s/it]

...

Error fetching data for PMID: 6928583
Error fetching data for PMID: 6928479
Error fetching data for PMID: 6928582
Error fetching data for PMID: 6928575
Error fetching data for PMID: 6928431
Error fetching data for PMID: 6928744
Error fetching data for PMID: 6767634
Error fetching data for PMID: 6774863
Error fetching data for PMID: 6772147
Error fetching data for PMID: 6928430
Error fetching data for PMID: 6928428
Error fetching data for PMID: 6928418
Error fetching data for PMID: 6928433
Error fetching data for PMID: 6771698
Error fetching data for PMID: 6928373
Error fetching data for PMID: 6769975
Error fetching data for PMID: 6777112
Error fetching data for PMID: 6774860
Error fetching data for PMID: 6928420
Error fetching data for PMID: 6928564
Error fetching data for PMID:Error fetching data for PMID: 6898937
Error fetching data for PMID: 6928372
 6755619
Error fetching data for PMID: 6766960
Error fetching data for PMID: 6446896
Error fetching data for PMID: 6767541
Error fetchi

  1%|          | 4/346 [05:09<7:17:39, 76.78s/it]

Error fetching data for PMID:Error fetching data for PMID: 6934114
Error fetching data for PMID: 6933766
 6934721
Error fetching data for PMID: 6933768
Error fetching data for PMID: 6934462
Error fetching data for PMID: 6934110
Error fetching data for PMID: 6933869
Error fetching data for PMID: 6934596
Error fetching data for PMID: 6935330
Error fetching data for PMID: 6935724
Error fetching data for PMID: 6933400
Error fetching data for PMID: 6933871
Error fetching data for PMID: 6933586
Error fetching data for PMID: 6934464
Error fetching data for PMID: 6933715
Error fetching data for PMID: 6933637
Error fetching data for PMID: 6933708
Error fetching data for PMID: 6933114
Error fetching data for PMID: 6933639
Error fetching data for PMID: 6933873
Error fetching data for PMID: 6932798
Error fetching data for PMID: 6933765
Error fetching data for PMID: 6933764
Error fetching data for PMID: 6933399
Error fetching data for PMID: 6933588
Error fetching data for PMID: 6933573
Error fetchi

  1%|          | 4/346 [06:17<8:58:12, 94.42s/it]


KeyboardInterrupt: 

In [10]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import time

def fetch_data(pmid):
    base_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator?pmids={}&concepts=Bioconcepts"
    url = base_url.format(pmid)
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        if response.status_code == 200:
            data = response.text.split('\n')
            if len(data) >= 2:
                title = data[0].split('|')[2].strip()
                abstract = data[1].split('|')[2].strip()
                return {'PubMed ID': pmid, 'Title': title, 'Abstract': abstract}
            else:
                print("No data found for PMID:", pmid)
        else:
            print("Error fetching data for PMID:", pmid)
    except requests.exceptions.RequestException as e:
        print(f"Request failed for PMID {pmid}: {e}")
    return None

# Define batch size
batch_size = 1000

# Split pmids into batches
pmids_batches = [pmids[i:i+batch_size] for i in range(0, len(pmids), batch_size)]

# Fetch data in parallel with tqdm progress tracking
results = []
with ThreadPoolExecutor(max_workers=10) as executor:
    for pmids_batch in tqdm(pmids_batches):
        batch_results = list(executor.map(fetch_data, pmids_batch))
        batch_results = [result for result in batch_results if result is not None]
        results.extend(batch_results)
        time.sleep(1)  # Add a small delay between requests to avoid overwhelming the server

# Create DataFrame from results
df_results = pd.DataFrame(results)

# Print the DataFrame
print(df_results)


  0%|          | 0/346 [00:00<?, ?it/s]

Request failed for PMID 7012186: 429 Client Error: Too Many Requests for url: https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator?pmids=7012186&concepts=Bioconcepts


  0%|          | 1/346 [00:48<4:39:59, 48.69s/it]

Request failed for PMID 6940951: 429 Client Error: Too Many Requests for url: https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator?pmids=6940951&concepts=Bioconcepts
Request failed for PMID 6933846: 429 Client Error: Too Many Requests for url: https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator?pmids=6933846&concepts=Bioconcepts


  1%|          | 3/346 [02:31<4:49:18, 50.61s/it]


KeyboardInterrupt: 

In [35]:
pmids=t_pmids
len(pmids)

345962

In [44]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import time

def fetch_data(pmids):
    base_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator?pmids={}&concepts=Bioconcepts"
    pmid_str = ','.join(map(str, pmids))  # Join PMIDs into a comma-separated string
    url = base_url.format(pmid_str)
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        if response.status_code == 200:
            # Split response into individual publications
            publications = response.text.split('\n\n')
            results = []
            for pub_data in publications:
                pub_lines = pub_data.split('\n')
                if len(pub_lines) >= 2:
                    pmid = pub_lines[0].split('|')[0].strip()
                    title = pub_lines[0].split('|')[2].strip()
                    abstract = pub_lines[1].split('|')[2].strip()
                    results.append({'PubMed ID': pmid, 'Title': title, 'Abstract': abstract})
                else:
                    print("No data found for PMID:", pmid)
            return results
        else:
            print("Error fetching data for PMIDs:", pmid_str)
    except requests.exceptions.RequestException as e:
        print(f"Request failed for PMIDs {pmid_str}: {e}")
    return []  # Return an empty list if no results are found

# Define batch size
batch_size = 100

# Split pmids into batches
pmids_batches = [pmids[i:i+batch_size] for i in range(0, len(pmids), batch_size)]

# Fetch data in parallel with tqdm progress tracking
results = []
with ThreadPoolExecutor(max_workers=20) as executor:
    for pmids_batch in tqdm(pmids_batches):
        batch_results = list(executor.map(fetch_data, [pmids_batch]))
        for result_list in batch_results:
            results.extend(result_list)
        time.sleep(0.5)  # Add a small delay between requests to avoid overwhelming the server

# Create DataFrame from results
df_results = pd.DataFrame(results)

# Print the DataFrame
print(df_results)


  0%|          | 0/3460 [00:00<?, ?it/s]

No data found for PMID: 31995917


  0%|          | 1/3460 [00:01<1:02:36,  1.09s/it]

No data found for PMID: 7229302


  0%|          | 2/3460 [00:02<1:02:03,  1.08s/it]

...

No data found for PMID: 38274045


 96%|█████████▌| 3318/3460 [1:06:01<02:50,  1.20s/it]

No data found for PMID: 38235441


 96%|█████████▌| 3319/3460 [1:06:02<02:48,  1.19s/it]


UnboundLocalError: local variable 'pmid' referenced before assignment

works relatively quickly but theres a problem with pmids fed to function

In [52]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import time

def fetch_data(pmids):
    base_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator"
    # Construct the request payload
    payload = {'pmids': pmids, 'concepts': 'Bioconcepts'}
    try:
        response = requests.post(base_url, data=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors
        if response.status_code == 200:
            # Split response into individual publications
            publications = response.text.split('\n\n')
            results = []
            for pub_data in publications:
                pub_lines = pub_data.split('\n')
                if len(pub_lines) >= 2:
                    pmid = pub_lines[0].split('|')[0].strip()
                    title = pub_lines[0].split('|')[2].strip()
                    abstract = pub_lines[1].split('|')[2].strip()
                    results.append({'PubMed ID': pmid, 'Title': title, 'Abstract': abstract})
                else:
                    print("No data found for PMID:", pmid)
            return results
        else:
            print("Error fetching data for PMIDs:", pmids)
    except requests.exceptions.RequestException as e:
        print(f"Request failed for PMIDs {pmids}: {e}")
    return []  # Return an empty list if no results are found

# Define batch size
batch_size = 999

# Split pmids into batches
pmids_batches = [pmids[i:i+batch_size] for i in range(0, len(pmids), batch_size)]

# Fetch data in parallel with tqdm progress tracking
results = []
with ThreadPoolExecutor(max_workers=10) as executor:
    for pmids_batch in tqdm(pmids_batches):
        batch_results = list(executor.map(fetch_data, [pmids_batch]))
        for result_list in batch_results:
            results.extend(result_list)
        time.sleep(2)  # Add a small delay between requests to avoid overwhelming the server

# Create DataFrame from results
df_results = pd.DataFrame(results)

# Print the DataFrame
print(df_results)


  0%|          | 0/347 [00:00<?, ?it/s]

No data found for PMID: 9


  0%|          | 1/347 [00:02<15:21,  2.66s/it]

No data found for PMID: 9


  1%|          | 2/347 [00:05<15:13,  2.65s/it]

...

No data found for PMID: 8


100%|█████████▉| 346/347 [15:15<00:02,  2.64s/it]

No data found for PMID: 8


100%|██████████| 347/347 [15:18<00:00,  2.65s/it]

     PubMed ID                                              Title Abstract
0            1  Formate assay in body fluids: application in m...         
1            3  Metal substitutions incarbonic anhydrase: a ha...         
2            4  Effect of chloroquine on cultured fibroblasts:...         
3            5  Atomic models for the polypeptide backbones of...         
4            6  Studies of oxygen binding energy to hemoglobin...         
...        ...                                                ...      ...
1777         2  Delineation of the intimate details of the bac...         
1778         3  Metal substitutions incarbonic anhydrase: a ha...         
1779         5  Atomic models for the polypeptide backbones of...         
1780         6  Studies of oxygen binding energy to hemoglobin...         
1781         8  Comparison between procaine and isocarboxazid ...         

[1782 rows x 3 columns]


In [53]:
df_results


,PubMed ID,Title,Abstract
0,1,Formate assay in body fluids: application in m...,
1,3,Metal substitutions incarbonic anhydrase: a ha...,
2,4,Effect of chloroquine on cultured fibroblasts:...,
3,5,Atomic models for the polypeptide backbones of...,
4,6,Studies of oxygen binding energy to hemoglobin...,
...,...,...,...
1777,2,Delineation of the intimate details of the bac...,
1778,3,Metal substitutions incarbonic anhydrase: a ha...,
1779,5,Atomic models for the polypeptide backbones of...,
1780,6,Studies of oxygen binding energy to hemoglobin...,


In [59]:
import pandas as pd
import requests
import time

def retrieve_pubtator_data(pmids, batch_size=100):
    # Define the PubTator API endpoint
    api_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator"
    
    # Initialize lists to store the results
    all_results = []
    
    # Split the list of PMIDs into batches
    batches = [pmids[i:i+batch_size] for i in range(0, len(pmids), batch_size)]
    
    # Loop through each batch of PMIDs
    for idx, batch in enumerate(batches, start=1):
        # Convert the list of PMIDs to a comma-separated string
        pmids_str = ','.join(batch)
        
        # Construct the payload for the POST request
        payload = {
            'pmids': pmids_str,
            'concepts': 'Bioconcepts'
        }
        
        try:
            # Send the POST request to the PubTator API
            response = requests.post(api_url, data=payload)
            
            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                # Save the batch of results to a temporary file
                with open(f'batch_{idx}.txt', 'w') as file:
                    file.write(response.text)
            else:
                print(f"Error for batch {idx}: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Error for batch {idx}: {e}")
        
        # Pause for 1/3 second to adhere to the rate limit
        time.sleep(1/3)
    
    # Concatenate all temporary files into a single dataframe
    df = pd.concat([pd.read_csv(f'batch_{idx}.txt', sep='\t', header=None) for idx in range(1, len(batches) + 1)], ignore_index=True)
    
    # Remove temporary files
    for idx in range(1, len(batches) + 1):
        file_name = f'batch_{idx}.txt'
        try:
            os.remove(file_name)
        except FileNotFoundError:
            pass
    
    return df

# Example list of PMIDs
pmids = t_pmids[200000:210000]

# Retrieve data from PubTator API in batches and store in a dataframe
pubtator_df = retrieve_pubtator_data(pmids)

# Display or further process the dataframe as needed
print(pubtator_df)


Error for batch 1: 400
Error for batch 2: 400
Error for batch 3: 400
Error for batch 4: 400
Error for batch 5: 400
Error for batch 6: 400
Error for batch 7: 400


KeyboardInterrupt: 

In [62]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import time

pmids = t_pmids[200000:210000]

def fetch_data(pmids):
    base_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator"
    # Construct the request payload
    payload = {'pmids': pmids, 'concepts': 'Bioconcepts'}
    try:
        response = requests.post(base_url, data=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors
        if response.status_code == 200:
            # Split response into individual publications
            publications = response.text.split('\n\n')
            results = []
            for pub_data in publications:
                pub_lines = pub_data.split('\n')
                if len(pub_lines) >= 2:
                    pmid = pub_lines[0].split('|')[0].strip()
                    title = pub_lines[0].split('|')[2].strip()
                    abstract = pub_lines[1].split('|')[2].strip()
                    results.append({'PubMed ID': pmid, 'Title': title, 'Abstract': abstract})
                else:
                    print("No data found for PMID:", pmid)
            return results
        else:
            print("Error fetching data for PMIDs:", pmids)
    except requests.exceptions.RequestException as e:
        print(f"Request failed for PMIDs {pmids}: {e}")
    return []  # Return an empty list if no results are found

# Define batch size
batch_size = 999

# Split pmids into batches
pmids_batches = [pmids[i:i+batch_size] for i in range(0, len(pmids), batch_size)]


# Fetch data in parallel with tqdm progress tracking
results = []
with ThreadPoolExecutor(max_workers=10) as executor:
    for pmids_batch in tqdm(pmids_batches):
        batch_results = list(executor.map(fetch_data, pmids_batch))  # Remove the extra list wrapping
        for result_list in batch_results:
            results.extend(result_list)
        time.sleep(2)  # Add a small delay between requests to avoid overwhelming the server

# Create DataFrame from results
df_results = pd.DataFrame(results)

# Print the DataFrame
print(df_results)


  0%|          | 0/11 [00:00<?, ?it/s]

No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data foun

  0%|          | 0/11 [00:07<?, ?it/s]


No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9
No data found for PMID: 9


KeyboardInterrupt: 

In [63]:
pmids

['19563047',
 '19563045',
 '19563044',
 '19563040',
 '19563039',
 '19563038',
 '19563037',
 '19543652',
 '19531166',
 '19522897',
 '19515023',
 '19489529',
 '19489527',
 '19489526',
 '19441525',
 '19441490',
 '19440411',
 '19424564',
 '19418899',
 '19418898',
 '19418897',
 '19418736',
 '19418735',
 '19410235',
 '19410234',
 '19410233',
 '19410232',
 '19410231',
 '19408689',
 '19402311',
 '19402310',
 '19402309',
 '19378635',
 '19371533',
 '19365897',
 '19358475',
 '19351015',
 '19347152',
 '19343930',
 '19343929',
 '19343928',
 '19343927',
 '19343924',
 '19343923',
 '19343922',
 '19331239',
 '19323410',
 '19323407',
 '19323406',
 '19323403',
 '19323401',
 '19323400',
 '19323399',
 '19323397',
 '19323396',
 '19323394',
 '19323393',
 '19323392',
 '19323389',
 '19323388',
 '19323387',
 '19323386',
 '19323385',
 '19323312',
 '19323311',
 '19323308',
 '19323307',
 '19323306',
 '19317203',
 '19317202',
 '19317201',
 '19301522',
 '19301521',
 '19301519',
 '19290439',
 '19290437',
 '19290436',

In [64]:
pmids_batch

['19563047',
 '19563045',
 '19563044',
 '19563040',
 '19563039',
 '19563038',
 '19563037',
 '19543652',
 '19531166',
 '19522897',
 '19515023',
 '19489529',
 '19489527',
 '19489526',
 '19441525',
 '19441490',
 '19440411',
 '19424564',
 '19418899',
 '19418898',
 '19418897',
 '19418736',
 '19418735',
 '19410235',
 '19410234',
 '19410233',
 '19410232',
 '19410231',
 '19408689',
 '19402311',
 '19402310',
 '19402309',
 '19378635',
 '19371533',
 '19365897',
 '19358475',
 '19351015',
 '19347152',
 '19343930',
 '19343929',
 '19343928',
 '19343927',
 '19343924',
 '19343923',
 '19343922',
 '19331239',
 '19323410',
 '19323407',
 '19323406',
 '19323403',
 '19323401',
 '19323400',
 '19323399',
 '19323397',
 '19323396',
 '19323394',
 '19323393',
 '19323392',
 '19323389',
 '19323388',
 '19323387',
 '19323386',
 '19323385',
 '19323312',
 '19323311',
 '19323308',
 '19323307',
 '19323306',
 '19317203',
 '19317202',
 '19317201',
 '19301522',
 '19301521',
 '19301519',
 '19290439',
 '19290437',
 '19290436',

In [66]:
pmids_batches

[['19563047',
  '19563045',
  '19563044',
  '19563040',
  '19563039',
  '19563038',
  '19563037',
  '19543652',
  '19531166',
  '19522897',
  '19515023',
  '19489529',
  '19489527',
  '19489526',
  '19441525',
  '19441490',
  '19440411',
  '19424564',
  '19418899',
  '19418898',
  '19418897',
  '19418736',
  '19418735',
  '19410235',
  '19410234',
  '19410233',
  '19410232',
  '19410231',
  '19408689',
  '19402311',
  '19402310',
  '19402309',
  '19378635',
  '19371533',
  '19365897',
  '19358475',
  '19351015',
  '19347152',
  '19343930',
  '19343929',
  '19343928',
  '19343927',
  '19343924',
  '19343923',
  '19343922',
  '19331239',
  '19323410',
  '19323407',
  '19323406',
  '19323403',
  '19323401',
  '19323400',
  '19323399',
  '19323397',
  '19323396',
  '19323394',
  '19323393',
  '19323392',
  '19323389',
  '19323388',
  '19323387',
  '19323386',
  '19323385',
  '19323312',
  '19323311',
  '19323308',
  '19323307',
  '19323306',
  '19317203',
  '19317202',
  '19317201',
  '193

In [67]:
import pandas as pd
import requests
from tqdm import tqdm
import time

# Assuming t_pmids is defined somewhere in your code
pmids = t_pmids[200000:210000]

def fetch_data(pmids):
    base_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator"
    results = []  # Initialize an empty list to store the results
    chunk_size = 999
    num_chunks = (len(pmids) + chunk_size - 1) // chunk_size  # Calculate the number of chunks
    for i in tqdm(range(num_chunks), desc="Fetching Data"):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, len(pmids))
        chunk_pmids = pmids[start_idx:end_idx]
        # Construct the request payload for each chunk of PMIDs
        payload = {'pmids': ','.join(chunk_pmids), 'concepts': 'Bioconcepts'}
        try:
            response = requests.post(base_url, data=payload)
            response.raise_for_status()  # Raise an exception for HTTP errors
            if response.status_code == 200:
                # Split response into individual publications
                publications = response.text.split('\n\n')
                for pub_data in publications:
                    pub_lines = pub_data.split('\n')
                    if len(pub_lines) >= 2:
                        # Extract PMID, Title, and Abstract from each publication
                        pmid = pub_lines[0].split('|')[0].strip()
                        title = pub_lines[0].split('|')[2].strip()
                        abstract = pub_lines[1].split('|')[2].strip()
                        results.append({'PubMed ID': pmid, 'Title': title, 'Abstract': abstract})
                    else:
                        print("No data found for PMID:", pmid)
            else:
                print("Error fetching data for PMIDs:", chunk_pmids)
        except requests.exceptions.RequestException as e:
            print(f"Request failed for PMIDs {chunk_pmids}: {e}")
        # Introduce a small delay to avoid overwhelming the server
        time.sleep(0.35)
    return results

# Call the fetch_data function and store the results
all_results = fetch_data(pmids)

# Convert the list of dictionaries into a pandas DataFrame
df = pd.DataFrame(all_results)

# Display the DataFrame
print(df.head())


Fetching Data:   0%|          | 0/11 [00:00<?, ?it/s]

Request failed for PMIDs ['19563047', '19563045', '19563044', '19563040', '19563039', '19563038', '19563037', '19543652', '19531166', '19522897', '19515023', '19489529', '19489527', '19489526', '19441525', '19441490', '19440411', '19424564', '19418899', '19418898', '19418897', '19418736', '19418735', '19410235', '19410234', '19410233', '19410232', '19410231', '19408689', '19402311', '19402310', '19402309', '19378635', '19371533', '19365897', '19358475', '19351015', '19347152', '19343930', '19343929', '19343928', '19343927', '19343924', '19343923', '19343922', '19331239', '19323410', '19323407', '19323406', '19323403', '19323401', '19323400', '19323399', '19323397', '19323396', '19323394', '19323393', '19323392', '19323389', '19323388', '19323387', '19323386', '19323385', '19323312', '19323311', '19323308', '19323307', '19323306', '19317203', '19317202', '19317201', '19301522', '19301521', '19301519', '19290439', '19290437', '19290436', '19290433', '19290432', '19290429', '19290428', '1

Fetching Data:   9%|▉         | 1/11 [00:00<00:08,  1.20it/s]

Request failed for PMIDs ['27672257', '27672256', '27672255', '27672254', '27672253', '27672251', '27672250', '27672249', '27672248', '27672247', '27672246', '27672245', '27672244', '21509774', '21492213', '21396603', '21396600', '21356010', '21195526', '21193170', '21190088', '21176870', '21158896', '21147150', '21146154', '21146152', '21145038', '21145037', '21144496', '21143533', '21138355', '21130420', '21127931', '21126218', '21122035', '21121960', '21114593', '21114591', '21114590', '21114589', '21112582', '21107784', '21088261', '21087316', '21087315', '21080788', '21080786', '21074463', '21073332', '21054511', '21054226', '21041048', '21038107', '21029265', '21029263', '21029149', '20969628', '20961345', '20961344', '20961342', '20952229', '20952225', '20945405', '20939845', '20936432', '20929720', '20927569', '20868433', '20849471', '20836083', '20836082', '20827678', '20801061', '20799278', '20677005', '20656517', '20638816', '20629090', '20572777', '20568035', '20434875', '2

Fetching Data:  18%|█▊        | 2/11 [00:01<00:07,  1.20it/s]

...

Request failed for PMIDs ['20156234', '20156233', '20154130', '20145963', '20144711', '20144538', '20129064', '20128827', '20127707', '20127504', '20127346', '20116355', '20113394', '20108107', '20102437', '20091849', '20091416', '20088703', '20087402', '20082174', '20081523', '20080960', '20080958', '20077466', '20074292', '20071132', '20067397', '20060730', '20059488', '20055661', '20053131', '20052608', '20043813', '20034012', '20034010', '20026290', '20025984', '20018797', '20014182', '20005619', '20002974', '19969525', '19967360', '19959612', '19959610', '19958168', '19957242', '19944782', '19941449', '19941294', '19925892', '19925891', '19921097', '19917626', '19913646', '19910208', '19909404', '19909403', '19909398', '19902322', '19879369', '19875744', '19860567', '19859733', '19854628', '19837747', '19793779', '19779767', '19763651', '19756402', '19755534', '19752016', '19745002', '19737135', '19729399', '19688227', '19660845', '19602052', '19592261', '19572921', '19557443', '1

Fetching Data:  91%|█████████ | 10/11 [00:08<00:00,  1.21it/s]

Request failed for PMIDs ['20608294', '20608293', '20606787', '20592834', '20578669', '20578668', '20578663', '20578662', '20578661', '20578660']: 400 Client Error: Bad Request for url: https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator


Fetching Data: 100%|██████████| 11/11 [00:09<00:00,  1.21it/s]

Empty DataFrame
Columns: []
Index: []


In [69]:
import pandas as pd
import requests
from tqdm import tqdm
import time

# Assuming t_pmids is defined somewhere in your code
pmids = t_pmids[200000:210000]

def fetch_data(pmids):
    base_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator"
    results = []  # Initialize an empty list to store the results
    chunk_size = 100
    num_chunks = (len(pmids) + chunk_size - 1) // chunk_size  # Calculate the number of chunks
    for i in tqdm(range(num_chunks), desc="Fetching Data"):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, len(pmids))
        chunk_pmids = pmids[start_idx:end_idx]
        for pmid in chunk_pmids:
            # Construct the request payload for each PMID individually
            payload = {'pmids': pmid, 'concepts': 'Bioconcepts'}
            try:
                response = requests.post(base_url, data=payload)
                response.raise_for_status()  # Raise an exception for HTTP errors
                if response.status_code == 200:
                    # Split response into individual publications
                    publications = response.text.split('\n\n')
                    for pub_data in publications:
                        pub_lines = pub_data.split('\n')
                        if len(pub_lines) >= 2:
                            # Extract PMID, Title, and Abstract from each publication
                            pmid = pub_lines[0].split('|')[0].strip()
                            title = pub_lines[0].split('|')[2].strip()
                            abstract = pub_lines[1].split('|')[2].strip()
                            results.append({'PubMed ID': pmid, 'Title': title, 'Abstract': abstract})
                        else:
                            print("No data found for PMID:", pmid)
                else:
                    print("Error fetching data for PMID:", pmid)
            except requests.exceptions.RequestException as e:
                print(f"Request failed for PMID {pmid}: {e}")
            # Introduce a small delay to avoid overwhelming the server
            time.sleep(1)
    return results

# Call the fetch_data function and store the results
all_results = fetch_data(pmids)

# Convert the list of dictionaries into a pandas DataFrame
df = pd.DataFrame(all_results)

# Display the DataFrame
print(df.head())


Fetching Data:   0%|          | 0/100 [00:00<?, ?it/s]

No data found for PMID: 9
No data found for PMID: 9


Fetching Data:   0%|          | 0/100 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [71]:
import pandas as pd
import requests
from tqdm import tqdm
import time

# Assuming t_pmids is defined somewhere in your code
pmids = t_pmids[200000:210000]

def fetch_data(pmids):
    base_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator"
    results = []  # Initialize an empty list to store the results
    chunk_size = 100
    num_chunks = (len(pmids) + chunk_size - 1) // chunk_size  # Calculate the number of chunks
    for i in tqdm(range(num_chunks), desc="Fetching Data"):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, len(pmids))
        chunk_pmids = pmids[start_idx:end_idx]
        # Construct the request payload for each chunk of 1000 PMIDs
        payload = {'pmids': ','.join(chunk_pmids), 'concepts': 'Bioconcepts'}
        try:
            response = requests.post(base_url, data=payload)
            response.raise_for_status()  # Raise an exception for HTTP errors
            if response.status_code == 200:
                # Split response into individual publications
                publications = response.text.split('\n\n')
                for pub_data in publications:
                    pub_lines = pub_data.split('\n')
                    if len(pub_lines) >= 2:
                        # Extract PMID, Title, and Abstract from each publication
                        pmid = pub_lines[0].split('|')[0].strip()
                        title = pub_lines[0].split('|')[2].strip()
                        abstract = pub_lines[1].split('|')[2].strip()
                        results.append({'PubMed ID': pmid, 'Title': title, 'Abstract': abstract})
                    else:
                        print("No data found for PMID:", pmid)
            else:
                print("Error fetching data for PMIDs:", chunk_pmids)
        except requests.exceptions.RequestException as e:
            print(f"Request failed for PMIDs {chunk_pmids}: {e}")
        # Introduce a small delay to avoid overwhelming the server
        time.sleep(0.1)
    return results

# Call the fetch_data function and store the results
all_results = fetch_data(pmids)

# Convert the list of dictionaries into a pandas DataFrame
df = pd.DataFrame(all_results)

# Display the DataFrame
print(df.head())


Fetching Data:   1%|          | 1/100 [00:00<00:55,  1.77it/s]

Request failed for PMIDs ['19563047', '19563045', '19563044', '19563040', '19563039', '19563038', '19563037', '19543652', '19531166', '19522897', '19515023', '19489529', '19489527', '19489526', '19441525', '19441490', '19440411', '19424564', '19418899', '19418898', '19418897', '19418736', '19418735', '19410235', '19410234', '19410233', '19410232', '19410231', '19408689', '19402311', '19402310', '19402309', '19378635', '19371533', '19365897', '19358475', '19351015', '19347152', '19343930', '19343929', '19343928', '19343927', '19343924', '19343923', '19343922', '19331239', '19323410', '19323407', '19323406', '19323403', '19323401', '19323400', '19323399', '19323397', '19323396', '19323394', '19323393', '19323392', '19323389', '19323388', '19323387', '19323386', '19323385', '19323312', '19323311', '19323308', '19323307', '19323306', '19317203', '19317202', '19317201', '19301522', '19301521', '19301519', '19290439', '19290437', '19290436', '19290433', '19290432', '19290429', '19290428', '1

Fetching Data:   2%|▏         | 2/100 [00:01<00:55,  1.76it/s]

Request failed for PMIDs ['19281111', '19281110', '19281109', '19281107', '19281106', '19281105', '19280910', '19280908', '19280905', '19280893', '19280891', '19280890', '19280889', '19280888', '19280736', '19278991', '19278986', '19278984', '19278983', '19278981', '19278979', '19276468', '19275088', '19275084', '19275081', '19275078', '19275077', '19254301', '19254300', '19248303', '19248301', '19247445', '19244887', '19244885', '19244882', '19244880', '19244878', '19241981', '19241980', '19239745', '19239744', '19239743', '19239742', '19239741', '19239740', '19239576', '19237382', '19237381', '19237379', '19237378', '19237377', '19237376', '19237373', '19237368', '19237367', '19237366', '19237365', '19237364', '19237363', '19237362', '19237361', '19237360', '19237359', '19237358', '19237357', '19237356', '19237355', '19237354', '19237353', '19237352', '19237351', '19237350', '19237349', '19237348', '19237347', '19237346', '19237025', '19235653', '19234079', '19234078', '19234077', '1

Fetching Data:   3%|▎         | 3/100 [00:01<00:55,  1.75it/s]

...

Fetching Data:  14%|█▍        | 14/100 [00:07<00:48,  1.76it/s]

Request failed for PMIDs ['21163625', '21156215', '21149622', '21145583', '21145210', '21144778', '21143530', '21142585', '21142583', '21133982', '21130183', '21129822', '21123030', '21122904', '21121988', '21121987', '21120558', '21118332', '21115324', '21112411', '21112362', '21111575', '21107594', '21106304', '21105846', '21098475', '21095133', '21091452', '21083570', '21075443', '21075442', '21074251', '21074250', '21074177', '21073982', '21068020', '21063810', '21062800', '21059105', '21056708', '21056408', '21055785', '21055726', '21054229', '21054199', '21050640', '21050381', '21050063', '21048794', '21043794', '21043792', '21041548', '21040809', '21039896', '21039628', '21039626', '21039623', '21035111', '21035110', '21029734', '20979990', '20969631', '20961685', '20961343', '20953743', '20952156', '20951243', '20949282', '20949276', '20936917', '20934794', '20933616', '20932158', '20931227', '20924603', '20924061', '20923449', '20923441', '20919950', '20880325', '20880136', '2

Fetching Data:  15%|█▌        | 15/100 [00:08<00:48,  1.75it/s]

Request failed for PMIDs ['20731735', '20731734', '20731733', '20722774', '20712437', '20674407', '20673636', '20662674', '20662665', '20652770', '20587692', '20587479', '20577772', '20554450', '20553166', '20553165', '20545550', '20500081', '20500054', '20433248', '20413167', '20406750', '20378189', '20373855', '20338971', '20336494', '20309597', '20309596', '20232086', '20221781', '20163254', '20162317', '20157753', '20151169', '20148278', '20142299', '20107826', '21237562', '23674910', '20950970', '20875936', '20851540', '21190990', '21146824', '21337175', '21147086', '21131362', '21213344', '21213342', '21344018', '21129747', '30828107', '25427329', '25427324', '21412843', '21332691', '21238790', '21235007', '21227568', '21214870', '21210503', '21210497', '21210488', '21210486', '21208207', '21205165', '21205164', '21199334', '21198899', '21195787', '21194947', '21193943', '21182947', '21181109', '21177012', '21173434', '21171159', '21169333', '21168349', '21168252', '21167759', '2

KeyboardInterrupt: 

worked, solution was to provide json 

In [92]:
import requests

# Define the base URL of the API
base_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator"

# Define the format, type, and list of PMIDs
pmids = ['23819905', '23819906', '23819907']  # Example list of PMIDs
payload = {'pmids': pmids, 'concepts': 'gene'}

try:
    # Send the POST request with JSON data
    response = requests.post(base_url, json=payload)
    response.raise_for_status()  # Raise an exception for HTTP errors
    if response.status_code == 200:
        # Print the response content
        print(response.text)
    else:
        print("Error: Request failed with status code:", response.status_code)
except requests.exceptions.RequestException as e:
    print("Request failed:", e)


23819905|t|Interrogating differences in expression of targeted gene sets to predict breast cancer outcome.
23819905|a|BACKGROUND: Genomics provides opportunities to develop precise tests for diagnostics, therapy selection and monitoring. From analyses of our studies and those of published results, 32 candidate genes were identified, whose expression appears related to clinical outcome of breast cancer. Expression of these genes was validated by qPCR and correlated with clinical follow-up to identify a gene subset for development of a prognostic test. METHODS: RNA was isolated from 225 frozen invasive ductal carcinomas,and qRT-PCR was performed. Univariate hazard ratios and 95% confidence intervals for breast cancer mortality and recurrence were calculated for each of the 32 candidate genes. A multivariable gene expression model for predicting each outcome was determined using the LASSO, with 1000 splits of the data into training and testing sets to determine predictive accuracy based o

In [ ]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import time

def fetch_data(pmids):
    base_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator"
    # Construct the request payload
    payload = {'pmids': pmids, 'concepts': 'Bioconcepts'}
    try:
        response = requests.post(base_url, data=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors
        if response.status_code == 200:
            # Split response into individual publications
            publications = response.text.split('\n\n')
            results = []
            for pub_data in publications:
                pub_lines = pub_data.split('\n')
                if len(pub_lines) >= 2:
                    pmid = pub_lines[0].split('|')[0].strip()
                    title = pub_lines[0].split('|')[2].strip()
                    abstract = pub_lines[1].split('|')[2].strip()
                    results.append({'PubMed ID': pmid, 'Title': title, 'Abstract': abstract})
                else:
                    print("No data found for PMID:", pmid)
            return results
        else:
            print("Error fetching data for PMIDs:", pmids)
    except requests.exceptions.RequestException as e:
        print(f"Request failed for PMIDs {pmids}: {e}")
    return []  # Return an empty list if no results are found

# Define batch size
batch_size = 999

# Split pmids into batches
pmids_batches = [pmids[i:i+batch_size] for i in range(0, len(pmids), batch_size)]

# Fetch data in parallel with tqdm progress tracking
results = []
with ThreadPoolExecutor(max_workers=10) as executor:
    for pmids_batch in tqdm(pmids_batches):
        batch_results = list(executor.map(fetch_data, [pmids_batch]))
        for result_list in batch_results:
            results.extend(result_list)
        time.sleep(2)  # Add a small delay between requests to avoid overwhelming the server

# Create DataFrame from results
df_results = pd.DataFrame(results)

# Print the DataFrame
print(df_results)


In [96]:
pmids = t_pmids

In [97]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import time

def fetch_data(pmids):
    base_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator"
    # Construct the request payload
    payload = {'pmids': pmids, 'concepts': 'Bioconcepts'}
    try:
        response = requests.post(base_url, json=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors
        if response.status_code == 200:
            # Split response into individual publications
            publications = response.text.split('\n\n')
            results = []
            for pub_data in publications:
                pub_lines = pub_data.split('\n')
                if len(pub_lines) >= 2:
                    pmid = pub_lines[0].split('|')[0].strip()
                    title = pub_lines[0].split('|')[2].strip()
                    abstract = pub_lines[1].split('|')[2].strip()
                    results.append({'PubMed ID': pmid, 'Title': title, 'Abstract': abstract})
                else:
                    print("No data found for PMID:", pmid)
            return results
        else:
            print("Error fetching data for PMIDs:", pmids)
    except requests.exceptions.RequestException as e:
        print(f"Request failed for PMIDs {pmids}: {e}")
    return []  # Return an empty list if no results are found

# Define batch size
batch_size = 999

# Split pmids into batches
pmids_batches = [pmids[i:i+batch_size] for i in range(0, len(pmids), batch_size)]

# Fetch data in parallel with tqdm progress tracking
results = []
with ThreadPoolExecutor(max_workers=10) as executor:
    for pmids_batch in tqdm(pmids_batches):
        batch_results = list(executor.map(fetch_data, [pmids_batch]))
        for result_list in batch_results:
            results.extend(result_list)
        time.sleep(2)  # Add a small delay between requests to avoid overwhelming the server

# Create DataFrame from results
df_results = pd.DataFrame(results)

# Print the DataFrame
print(df_results)


  0%|          | 0/347 [00:00<?, ?it/s]

No data found for PMID: 31995917


  0%|          | 1/347 [00:02<16:18,  2.83s/it]

No data found for PMID: 30170475


  1%|          | 2/347 [00:05<16:20,  2.84s/it]

...

No data found for PMID: 38293571


100%|█████████▉| 346/347 [17:20<00:02,  2.96s/it]

No data found for PMID: 37488860


100%|██████████| 347/347 [17:23<00:00,  3.01s/it]


       PubMed ID                                              Title  \
0        6108426                    Dental caries and malnutrition.   
1        6110572  Separation of inorganic pyrophosphatase from a...   
2        6112979                             The knocked out tooth.   
3        6113713  Are biomedical exchange programs inside The Pe...   
4        6114139                           Pediatric dental trauma.   
...          ...                                                ...   
331308  37471164  The Root Complex in Posterior Teeth: A Direct ...   
331309  37471297  Evaluation of the efficacy of hydroxyl radical...   
331310  37475564  Non-invasive and Locally Invasive Hypercellula...   
331311  37483041  Exploring the aspirations, expectations and im...   
331312  37488860  Beyond dentistry: could prevention and screeni...   

                                                 Abstract  
0                                                          
1                          

In [98]:
df_results

,PubMed ID,Title,Abstract
0,6108426,Dental caries and malnutrition.,
1,6110572,Separation of inorganic pyrophosphatase from a...,
2,6112979,The knocked out tooth.,
3,6113713,Are biomedical exchange programs inside The Pe...,Between November 1979 and April 1980 we lived ...
4,6114139,Pediatric dental trauma.,
...,...,...,...
331308,37471164,The Root Complex in Posterior Teeth: A Direct ...,The aim of this morphometric study was to prec...
331309,37471297,Evaluation of the efficacy of hydroxyl radical...,BACKGROUND: Concerning about the quality of ro...
331310,37475564,Non-invasive and Locally Invasive Hypercellula...,Medical records of dogs with an initial histop...
331311,37483041,"Exploring the aspirations, expectations and im...",BACKGROUND: Local anaesthesia is the anaesthet...


In [99]:
df_results.to_csv('title_abstract.tsv', sep='\t', index=False)
